# Site App with Database 

* About Me Section
* Notepad: Create, Read
* Counter

## Create a database server using Docker

### Postgres Docker

In [ ]:
# Without volume/data connected
!docker run --name postgres-sql -e POSTGRES_USER=matt -e POSTGRES_PASSWORD=password  -d postgres

In [ ]:
# With volume/data connected
!docker run -d \
    --name postgres-sql \
    -e POSTGRES_USER=matt \
    -e POSTGRES_PASSWORD=password \
    -v $(pwd)/data/:/var/lib/postgresql/data/ \
    -p 5000:5432 \
    postgres

In [ ]:
!docker ps

In [ ]:
!ls -a

In [ ]:
# Check the data
# In a terminal
# sudo su
# cd $(pwd)/data/
# ls -a

In [ ]:
# Test the data of the DB
# docker rm -f <CONTAINER_ID>
# run the docker run <...>
# check the pgAdmin
# the data persists

### pgAdmin

In [ ]:
!docker run -p 8000:80 \
    -e 'PGADMIN_DEFAULT_EMAIL=matt@example.com' \
    -e 'PGADMIN_DEFAULT_PASSWORD=password' \
    -d dpage/pgadmin4

In [ ]:
!docker ps

### Create a firewall (GCP)

In [ ]:
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:5401,tcp:9000 --source-ranges=0.0.0.0/0

### Open pgAdmin Browser

In [ ]:
# http://<IP_ADDRESS>:<PORT>
# Login with credentials

# Register Server
# HOST: <IP_ADDRESS>
# PORT: 5000
# USER: <POSTGRES_USER>
# PASSWORD: <PASSWORD>

## Install and import libraries

In [ ]:
!pip install streamlit psycopg2-binary -q

In [ ]:
import streamlit as st
import psycopg2